In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 6

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 12

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
19686,"[1675.693359375, 301.4346923828125]","[1676.3175048828125, 302.0689392089844]","[1677.1024169921875, 302.864013671875]","[1677.8592529296875, 303.6157531738281]","[1678.6302490234375, 304.3883361816406]","[1679.3946533203125, 305.1500549316406]","[1680.156982421875, 305.90545654296875]","[1680.916748046875, 306.6522216796875]","[1681.675048828125, 307.3909912109375]","[1682.4332275390625, 308.1226501464844]",...,"[1691.013427734375, 316.1467590332031]","[1691.8212890625, 316.906005859375]","[1692.6309814453125, 317.670654296875]","[1693.4404296875, 318.4388122558594]","[1694.247314453125, 319.20782470703125]","[1695.0489501953125, 319.974365234375]","[1695.842041015625, 320.7344055175781]","[1696.6234130859375, 321.4835205078125]","[1697.3895263671875, 322.21697998046875]","[1698.136962890625, 322.93017578125]"
25066,"[2179.39501953125, 720.4036254882812]","[2179.9453125, 720.7996826171875]","[2180.740966796875, 721.5216064453125]","[2181.47607421875, 722.1550903320312]","[2182.22705078125, 722.8236694335938]","[2182.962158203125, 723.4776611328125]","[2183.6884765625, 724.1259155273438]","[2184.405029296875, 724.7649536132812]","[2185.11376953125, 725.3955078125]","[2185.81640625, 726.0181884765625]",...,"[2193.53564453125, 732.7665405273438]","[2194.260986328125, 733.4091186523438]","[2194.991455078125, 734.0607299804688]","[2195.7265625, 734.72119140625]","[2196.46533203125, 735.3898315429688]","[2197.2060546875, 736.0653076171875]","[2197.947509765625, 736.7456665039062]","[2198.68701171875, 737.42822265625]","[2199.42236328125, 738.1097412109375]","[2200.150634765625, 738.7864379882812]"
9016,"[238.63555908203125, 1357.744873046875]","[239.124267578125, 1355.66845703125]","[238.95347595214844, 1354.3818359375]","[239.0093994140625, 1352.9842529296875]","[239.0745849609375, 1351.5081787109375]","[239.13560485839844, 1350.04443359375]","[239.19293212890625, 1348.612548828125]","[239.24627685546875, 1347.2113037109375]","[239.29550170898438, 1345.8377685546875]","[239.3406982421875, 1344.488525390625]",...,"[239.64886474609375, 1330.583251953125]","[239.66693115234375, 1329.36083984375]","[239.6841278076172, 1328.1402587890625]","[239.70040893554688, 1326.9205322265625]","[239.71566772460938, 1325.700927734375]","[239.72967529296875, 1324.4808349609375]","[239.74209594726562, 1323.2596435546875]","[239.7523956298828, 1322.0372314453125]","[239.75991821289062, 1320.813232421875]","[239.76376342773438, 1319.587890625]"
26761,"[1886.2054443359375, 480.8128356933594]","[1887.4246826171875, 481.84771728515625]","[1888.583984375, 482.7347412109375]","[1889.834228515625, 483.75250244140625]","[1891.08837890625, 484.79010009765625]","[1892.3447265625, 485.8416442871094]","[1893.601806640625, 486.90045166015625]","[1894.859375, 487.96246337890625]","[1896.1175537109375, 489.025390625]","[1897.3778076171875, 490.08917236328125]",...,"[1911.2178955078125, 501.9750061035156]","[1912.33984375, 502.9377136230469]","[1913.4251708984375, 503.86895751953125]","[1914.4906005859375, 504.7947998046875]","[1915.5584716796875, 505.75048828125]","[1916.642822265625, 506.7596435546875]","[1917.731689453125, 507.8072204589844]","[1918.788818359375, 508.8397216796875]","[1919.7783203125, 509.8014831542969]","[1920.684326171875, 510.6632385253906]"
24219,"[1731.974365234375, 359.3661804199219]","[1731.3641357421875, 358.9465637207031]","[1730.5819091796875, 358.3211669921875]","[1729.91845703125, 357.7038269042969]","[1729.2830810546875, 357.0933532714844]","[1728.6416015625, 356.49005126953125]","[1727.9974365234375, 355.8890686035156]","[1727.35107421875, 355.2914733886719]","[1726.70263671875, 354.6974792480469]","[1726.0521240234375, 354.106689453125]",...,"[1718.7425537109375, 347.69940185546875]","[1718.0650634765625, 347.11700439453125]","[1717.3863525390625, 346.53399658203125]","[1716.7066650390625, 345.95037841796875]","[1716.026611328125, 345.36614990234375]","[1715.346435546875,

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
19686,1675.693359,301.434692,1676.317505,302.068939,1677.102417,302.864014,1677.859253,303.615753,1678.630249,304.388336,...,1695.048950,319.974365,1695.842041,320.734406,1696.623413,321.483521,1697.389526,322.216980,1698.136963,322.930176
25066,2179.395020,720.403625,2179.945312,720.799683,2180.740967,721.521606,2181.476074,722.155090,2182.227051,722.823669,...,2197.206055,736.065308,2197.947510,736.745667,2198.687012,737.428223,2199.422363,738.109741,2200.150635,738.786438
9016,238.635559,1357.744873,239.124268,1355.668457,238.953476,1354.381836,239.009399,1352.984253,239.074585,1351.508179,...,239.729675,1324.480835,239.742096,1323.259644,239.752396,1322.037231,239.759918,1320.813232,239.763763,1319.587891
26761,1886.205444,480.812836,1887.424683,481.847717,1888.583984,482.734741,1889.834229,483.752502,1891.088379,484.790100,...,1916.642822,506.759644,1917.731689,507.807220,1918.788818,508.839722,1919.778320,509.801483,1920.684326,510.663239
24219,1731.974365,359.366180,1731.364136,358.946564,1730.581909,358.321167,1729.918457,357.703827,1729.283081,357.093353,...,1715.346436,344.781342,1714.666870,344.195984,1713.988281,343.610168,1713.311157,343.023804,1712.636108,342.436920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35281,265.885040,1028.415283,266.494141,1028.610474,266.841949,1028.247437,267.327026,1028.106201,267.841003,1028.012939,...,281.702576,1029.158447,282.265594,1029.309937,282.809662,1029.464478,283.335052,1029.621704,283.842041,1029.781372
31996,1741.607300,450.228882,1741.739990,449.950287,1741.916382,449.507233,1742.192139,449.140991,1742.489990,448.815369,...,1754.767212,445.400757,1755.284424,445.396973,1755.778931,445.397278,1756.251343,445.401031,1756.702637,445.407654
27791,589.284912,1386.227905,589.791077,1385.711914,589.254211,1386.002930,588.867554,1385.904907,588.529846,1385.853882,...,582.489258,1385.159668,582.193604,1385.105347,581.892822,1385.041870,581.586731,1384.969360,581.275330,1384.887939


In [12]:
df2.to_csv("outputs12epb6.csv", index=True)